### CS 686 Assignment 3

##### Submitted by: Priyansh Narang, 20716980

General Note: 
1. I have used integer representation to model the boolean random variables. So, true = 1 and false = 0.
2. I used dataframes in pandas deliberately to increase readability. 
3. I have tried wherever possible to keep the variables in an alphabetical order. You might encounter a few anamolies but I hope they are not that bad

In [2]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
from collections import OrderedDict

In [3]:
# restrictedFactor = restrict(factor, variable, value): This function restricts
# the given variable in the given factor to the given value. [Tip: use slicing operations
# to implement this function.]

def restrict(factor, variable, value):
    
    # check if there is a value for every varible
    assert(len(variable) == len(value))
    
    for idx, cur_var in enumerate(variable):
        factor = factor[factor.loc[:, cur_var]==value[idx]].loc[:, factor.columns != cur_var]
    
    return factor

In [4]:
factor = pd.DataFrame([[1,1,1,0.1],[1,1,0,0.9],[1,0,1,0.2],[1,0,0,0.8],[0,1,1,0.4],[0,1,0,0.6],[0,0,1,0.3],[0,0,0,0.7]], columns=['X','Y', 'Z', 'val'])
variable = ['X']
value = [1]

In [5]:
f1 = restrict(factor, ['X'], [1])
f2 = restrict(factor, ['Y'], [0])
print(f1)
print(f2)

   Y  Z  val
0  1  1  0.1
1  1  0  0.9
2  0  1  0.2
3  0  0  0.8
   X  Z  val
2  1  1  0.2
3  1  0  0.8
6  0  1  0.3
7  0  0  0.7


In [271]:
# productFactor = multiply(factor1, factor2): This function performs pointwise
# multiplication of the given two factors. [Tip: take advantage of the numpy
# broadcasting rules to multiply multi-dimensional arrays of different shapes.]


# f3 = pd.merge(f1, f2, on='B',how='outer', validate = "many_to_many")
def multiply(factor1, factor2):
    if factor2 is None:
        return factor1
    
    elif factor1 is None:
        return factor2
    
    columns = sorted((set(factor1.columns[:-1]) & set(factor2.columns[:-1])))
    
    #print(factor1)
    #print(factor2)
    
    # When either one of them is scalar, factor2 is sent to be a scalar
    if (len(columns) == 0) & (len(factor2.columns[:-1])==0):
        factor1.loc[:, 'val'] = factor1["val"] * factor2["val"].item()
        return factor1
    
    if (len(columns) == 0) & (len(factor1.columns[:-1])==0):
        factor2.loc[:, 'val'] = factor2["val"] * factor1["val"].item()
        return factor2
    
    df = pd.merge(factor1, factor2, on=columns, how='inner')
    df['val'] = df.val_x * df.val_y
    df.drop(['val_x', 'val_y'], axis=1, inplace = True)
    return df

In [7]:
productFactor = multiply(f1, f2)
print(productFactor)

   Y  Z  X   val
0  1  1  1  0.02
1  1  1  0  0.03
2  0  1  1  0.04
3  0  1  0  0.06
4  1  0  1  0.72
5  1  0  0  0.63
6  0  0  1  0.64
7  0  0  0  0.56


In [8]:
# resultFactor = sumout(factor, variable): This function sums out a variable
# in a given factor. [Tip: use the sum operation to implement this function.]

def sumout(factor, variable):
    variable.append('val')
    grouping_columns = list(factor.columns.difference(variable))
    if len(grouping_columns) == 0:
        return None
    df = factor.groupby(grouping_columns, as_index=False).sum()
    df.drop(variable[0], axis=1, inplace=True)
    return df

In [9]:
sumout(f1, ['Y'])

,Z,val
0,0,1.7
1,1,0.3


In [10]:
# normalizedFactor = normalize(factor): This function normalizes a factor by
# dividing each entry by the sum of all. the entries. This function is useful when
# the factor is a distribution (i.e. the sum of the probabilities must be 1.).

def normalize(factor):
    """
    Normalize a factor
    
    Sum each column and divide individual elements in that column by the sum.
    """
    normalized_sum = factor.sum(axis=0)
    normlaized_factor = []
    
    if normalized_sum.all() == 1:
        normlaized_factor = factor.divide(normalized_sum, axis=1)
        
    return normlaized_factor

In [11]:
normalize(sumout(f1, ['Y']))

,Z,val
0,0.0,0.85
1,1.0,0.15


In [229]:
def inference(factor_list, query_variables, order_list_of_hidden_variables, evidence_variables):
    """
    Compute Pr(query_variables|evidence_variables) following the variable elimination algorithm: 
    
    Steps taken
    1. Restrict the factors in factor_list according to the evidence in evidence_variables
    2. Based on the order in order_list_of_hidden_Variables, sum out each hidden variable from the product of factors in the factor list
    3. If the result is a product of multiple factors, multiply them to produce a single factor.
    4. If the resulting factor should denote a probability distribution, normalize the resulting factor.
    """
    
    #Step 1: Restrict. 
    #For each evidence variable pick up the factor that has that variable and restrict the rows to the value
    for eidx, evar in enumerate(evidence_variables):
        for fidx, fvar in enumerate(factor_list):
            if evar in fvar.columns:
                factor_list[fidx] = restrict(fvar, [evar], [1])
        
    
    #Step 2: Multiply and sum out
    #Go over all the variables in the order of variable elimination. For each, get the factors relevant to that variable by
    #querying the dictionary, multiply them incrementally and sum them out
    #delete the older factors from the dictionary 
    #update the dictionary of factor list with the new factor as this will may be picked up by another variable later. 
    
    
    #flist = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
    #hidden_variables = ['M']

    for hvaridx, hvar in enumerate(hidden_variables):
        temp_list = []
        fidx = 0
        while fidx < len(factor_list):
            if hvar in factor_list[fidx].columns: 
                temp_list.append(factor_list.pop(fidx))
                fidx -= 1
            fidx += 1

        #print(temp_list)

        sumout_factor_temp = temp_list[0]
        templist_idx = 1

        while ((len(temp_list) > 1) & (templist_idx < len(temp_list))):
            sumout_factor_temp = multiply(sumout_factor_temp, temp_list[templist_idx])
            templist_idx += 1
            #print(templist_idx)

        sumout_factor = sumout(sumout_factor_temp, [hvar])
        #print(sumout_factor)
        factor_list.append(sumout_factor)
        
    #Step 3
    #If the result is more than one factor we multiply to produce a single factor
    result_factor_temp = factor_list[0]
    fidx = 1
    
    while ((len(factor_list) > 1) & (fidx < len(factor_list))):
        result_factor_temp = multiply(result_factor_temp, factor_list[fidx])
        fidx += 1
    
    #Step 4:
    #Normalize the result factor and return it
    result_factor = normalize(result_factor_temp)
    print(result_factor)
    
    


## Exercise 1

**Q2b** 
1. P(FH)
2. Query Variables: FH
3. Evidence Variables: None
4. Hidden variables: everything else
<br/>
Using the bayesian network we are able to reach the equation:


$$\sum_B \sum_M \sum_{NA} \sum_{NH} \sum_S P(B=b\mid S=s) P(M=m) P(NA=na) P(NH=nh \mid M=m \wedge NA=na) P(S=s) P(FH=t \mid M=m \wedge NH=nh \wedge S=s) $$

Define a factor for $$ f_1(B,S) = P(B=b\mid S=s)$$

In [287]:
f1_BS = pd.DataFrame([[1,1,0.6],[1,0,0.1],[0,1,0.4],[0,0,0.9]], columns =['B', 'S', 'val'])
f1_BS

,B,S,val
0,1,1,0.6
1,1,0,0.1
2,0,1,0.4
3,0,0,0.9


**Define a factor for** $$ f_2(M) =  P(M=m)$$

In [288]:
f2_M = pd.DataFrame([[1, 1./28], [0, 27./28]], columns = ['M', 'val'])
f2_M

,M,val
0,1,0.035714
1,0,0.964286


**Define a factor for** $$ f_3(NA) = P(NA=na)$$

In [289]:
f3_NA = pd.DataFrame([[1, 0.3], [0,0.7]], columns = ['NA', 'val'])
f3_NA

,NA,val
0,1,0.3
1,0,0.7


**Define a factor for** $$ f_4(M,NA,NH) =  P(NH=nh \mid M=m \wedge NA=na)$$

In [290]:
f4_MNANH = pd.DataFrame([[1,1,1,0.8],
                        [1,1,0,0.2],
                        [1,0,1,0.4],
                        [1,0,0,0.6],
                        [0,1,1,0.5],
                        [0,1,0,0.5],
                        [0,0,1,0.0],
                        [0,0,0,1.0]], columns = ['M','NA','NH','val'])
f4_MNANH

,M,NA,NH,val
0,1,1,1,0.8
1,1,1,0,0.2
2,1,0,1,0.4
3,1,0,0,0.6
4,0,1,1,0.5
5,0,1,0,0.5
6,0,0,1,0.0
7,0,0,0,1.0


**Define a factor for** $$ f_5(S) =  P(S=s)$$

In [291]:
f5_S = pd.DataFrame([[1,0.05], [0,0.95]], columns = ['S', 'val'])
f5_S

,S,val
0,1,0.05
1,0,0.95


**Define a factor for** $$ f_6(FH,M,NH,S) =  P(FH=t \mid M=m \wedge NH=nh \wedge S=s)$$

In [292]:
f6_FHMNHS = pd.DataFrame([[1,1,1,1,0.99],
                         [1,1,1,0,0.65],
                         [1,1,0,1,0.90],
                         [1,1,0,0,0.40],
                         [1,0,1,1,0.75],
                         [1,0,1,0,0.20],
                         [1,0,0,1,0.50],
                         [1,0,0,0,0.00],
                         [0,1,1,1,0.01],
                         [0,1,1,0,0.35],
                         [0,1,0,1,0.10],
                         [0,1,0,0,0.60],
                         [0,0,1,1,0.25],
                         [0,0,1,0,0.80],
                         [0,0,0,1,0.50],
                         [0,0,0,0,1.00]], columns = ['FH', 'M', 'NH', 'S', 'val'])
f6_FHMNHS

,FH,M,NH,S,val
0,1,1,1,1,0.99
1,1,1,1,0,0.65
2,1,1,0,1,0.90
3,1,1,0,0,0.40
4,1,0,1,1,0.75
5,1,0,1,0,0.20
6,1,0,0,1,0.50
7,1,0,0,0,0.00
8,0,1,1,1,0.01
9,0,1,1,0,0.35


So, now we can represent the equation we need as: 

$$\sum_B \sum_M \sum_{NA} \sum_{NH} \sum_S f_1(B,S) \times f_2(M) \times \times f_3(NA) \times f_4(M,NA,NH)\times f_5(S)\times  f_6(FH,M,NH,S) $$


In [224]:
hidden_variables = ['B', 'M', 'NA', 'NH', 'S']
# factor_list = OrderedDict().fromkeys(hidden_variables)
# factor_list['B'] = [f1_BS]
# factor_list['FH'] = [f6_FHMNHS]
# factor_list['M'] = [f2_M, f4_MNANH, f6_FHMNHS]
# factor_list['NA'] = [f3_NA, f4_MNANH]
# factor_list['NH'] = [f4_MNANH, f6_FHMNHS]
# factor_list['S'] = [f5_S, f6_FHMNHS]
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
query_variables = ['FH']
evidence_variables = []

In [226]:
inference(factor_list,query_variables, hidden_variables, evidence_variables)

    FH      val
0  0.0  0.92693
1  1.0  0.07307


In [256]:
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]

In [318]:
factor_list

[   S  val
 0  1  0.6
 1  0  0.1,         val
 0  0.035714,    S   val
 0  1  0.05
 1  0  0.95,    S     val
 0  0  0.5300
 1  1  0.9468]

In [266]:
cp_factor_list = factor_list

In [319]:
result_factor_temp = factor_list[0]
fidx = 1

while ((len(factor_list) > 1) & (fidx < len(factor_list))):
    result_factor_temp = multiply(result_factor_temp, factor_list[fidx])
    fidx += 1

In [320]:
result_factor_temp

,S,val
0,1,0.001014
1,0,0.001798


In [309]:
#factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
hidden_variables = ['NA', 'NH']
#hidden_variables = ['B', 'NA', 'NH']

for hvaridx, hvar in enumerate(hidden_variables):
    temp_list = []
    fidx = 0
    while fidx < len(factor_list):
        print(factor_list[fidx].columns)
        if hvar in factor_list[fidx].columns:
            print(fidx)
            temp_list.append(factor_list.pop(fidx))
            fidx -= 1
        fidx += 1

    #print(temp_list)
    
    sumout_factor_temp = temp_list[0]
    templist_idx = 1

    while ((len(temp_list) > 1) & (templist_idx < len(temp_list))):
        sumout_factor_temp = multiply(sumout_factor_temp, temp_list[templist_idx])
        templist_idx += 1
        #print(templist_idx)
        
    sumout_factor = sumout(sumout_factor_temp, [hvar])
    print(sumout_factor)
    factor_list.append(sumout_factor)


Index(['S', 'val'], dtype='object')
Index(['NH', 'S', 'val'], dtype='object')
Index(['val'], dtype='object')
Index(['NA', 'NH', 'val'], dtype='object')
3
Index(['NA', 'val'], dtype='object')
3
Index(['S', 'val'], dtype='object')
   NH   val
0   0  0.48
1   1  0.52
Index(['S', 'val'], dtype='object')
Index(['NH', 'S', 'val'], dtype='object')
1
Index(['val'], dtype='object')
Index(['S', 'val'], dtype='object')
Index(['NH', 'val'], dtype='object')
3
   S     val
0  0  0.5300
1  1  0.9468


In [164]:
flist = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
hidden_variables = ['S']

for hvaridx, hvar in enumerate(hidden_variables):
    temp_list = []
    fidx = 0
    while fidx < len(flist):
        if hvar in flist[fidx].columns: #fvar.columns:
            temp_list.append(flist.pop(fidx))
            fidx -= 1
        fidx += 1


[   B  S  val
0  1  1  0.6
1  1  0  0.1
2  0  1  0.4
3  0  0  0.9,     FH  M  NH  S   val
0    1  1   1  1  0.99
1    1  1   1  0  0.65
2    1  1   0  1  0.90
3    1  1   0  0  0.40
4    1  0   1  1  0.75
5    1  0   1  0  0.20
6    1  0   0  1  0.50
7    1  0   0  0  0.00
8    0  1   1  1  0.01
9    0  1   1  0  0.35
10   0  1   0  1  0.10
11   0  1   0  0  0.60
12   0  0   1  1  0.25
13   0  0   1  0  0.80
14   0  0   0  1  0.50
15   0  0   0  0  1.00,    S   val
0  1  0.05
1  0  0.95]
---------------------
[   M       val
0  1  0.035714
1  0  0.964286,    M  NA  NH  val
0  1   1   1  0.8
1  1   1   0  0.2
2  1   0   1  0.4
3  1   0   0  0.6
4  0   1   1  0.5
5  0   1   0  0.5
6  0   0   1  0.0
7  0   0   0  1.0,    NA  val
0   1  0.3
1   0  0.7]


In [205]:
evidence_variables = ['S', 'M']

In [280]:
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]

In [307]:
factor_list

[   S  val
 0  1  0.6
 1  0  0.1,    NH  S   val
 0   1  1  0.99
 1   1  0  0.65
 2   0  1  0.90
 3   0  0  0.40,         val
 0  0.035714,    NA  NH  val
 0   1   1  0.8
 1   1   0  0.2
 2   0   1  0.4
 3   0   0  0.6,    NA  val
 0   1  0.3
 1   0  0.7,    S       val
 0  1  0.000064
 1  0  0.001212]

In [282]:
evidence_variables = ['B','FH', 'M']
print(factor_list)
for eidx, evar in enumerate(evidence_variables):
    for fidx, fvar in enumerate(factor_list):
        if evar in fvar.columns:
            factor_list[fidx] = restrict(fvar, [evar], [1])
    print(factor_list)
    print("-----------------------------------------")

[   B  S  val
0  1  1  0.6
1  1  0  0.1
2  0  1  0.4
3  0  0  0.9,     FH  M  NH  S   val
0    1  1   1  1  0.99
1    1  1   1  0  0.65
2    1  1   0  1  0.90
3    1  1   0  0  0.40
4    1  0   1  1  0.75
5    1  0   1  0  0.20
6    1  0   0  1  0.50
7    1  0   0  0  0.00
8    0  1   1  1  0.01
9    0  1   1  0  0.35
10   0  1   0  1  0.10
11   0  1   0  0  0.60
12   0  0   1  1  0.25
13   0  0   1  0  0.80
14   0  0   0  1  0.50
15   0  0   0  0  1.00,    M       val
0  1  0.035714
1  0  0.964286,    M  NA  NH  val
0  1   1   1  0.8
1  1   1   0  0.2
2  1   0   1  0.4
3  1   0   0  0.6
4  0   1   1  0.5
5  0   1   0  0.5
6  0   0   1  0.0
7  0   0   0  1.0,    NA  val
0   1  0.3
1   0  0.7,    S       val
0  1  0.000064
1  0  0.001212]
[   S  val
0  1  0.6
1  0  0.1,     FH  M  NH  S   val
0    1  1   1  1  0.99
1    1  1   1  0  0.65
2    1  1   0  1  0.90
3    1  1   0  0  0.40
4    1  0   1  1  0.75
5    1  0   1  0  0.20
6    1  0   0  1  0.50
7    1  0   0  0  0.00
8    0  1   1

In [21]:
for idx, var in enumerate(evidence_variables):
    print(var)
    var_factors = factor_list[var]
    for vidx, fvar in enumerate(var_factors):
        var_factors[vidx] = restrict(fvar, [var], [1])
        
    factor_list[var] = var_factors

S


In [22]:
var_factors = factor_list['B']
sumout_factor_temp = var_factors[0]
fidx = 1

while len(var_factors) > 1 & fidx < len(var_factors):
    sumout_factor_temp = multiply(sumout_factor_temp, var_factors[fidx])
    fidx += 1

sumout_factor = sumout(sumout_factor_temp, ['B'])
sumout_factor

,S,val
0,0,1.0
1,1,1.0


In [172]:
flist

[   FH      val
 0   0  0.92693
 1   1  0.07307]

** Step 2: Restrict the variables ** 
<br/>
<br/>
Since, we don't observe anything in this case our evidence variables is an empty set 

**Step 3: Sum out the hidden variables according to an order**
<br/>
<br/>
Order as given in the question is:
$$B,FH, M, NA, NH, S$$

**B**

In [24]:
f7_S = sumout(f1_BS, ['B'])
f7_S

,S,val
0,0,1.0
1,1,1.0


**Updated Expression:** $$\sum_M \sum_{NA} \sum_{NH} \sum_S f_2(M) \times f_3(NA) \times f_4(M,NA,NH)\times f_5(S)\times  f_6(FH,M,NH,S) \times f_7(S) $$

**Next Variable: M**

In [25]:
f8_FHMNANHS_temp = multiply(f2_M,f4_MNANH)
f8_FHMNANHS = multiply(f8_FHMNANHS_temp, f6_FHMNHS)
f8_FHMNANHS.head()

,M,NA,NH,FH,S,val
0,1,1,1,1,1,0.028286
1,1,1,1,1,0,0.018571
2,1,1,1,0,1,0.000286
3,1,1,1,0,0,0.010000
4,1,0,1,1,1,0.014143


In [26]:
f9_FHNANHS = sumout(f8_FHMNANHS, ['M'])
f9_FHNANHS.head()

,FH,NA,NH,S,val
0,0,0,0,0,0.977143
1,0,0,0,1,0.484286
2,0,0,1,0,0.005000
3,0,0,1,1,0.000143
4,0,1,0,0,0.486429


**Updated Expression:** $$\sum_{NA} \sum_{NH} \sum_S f_3(NA) \times f_5(S) \times f_7(S) \times f_9(FH,NA,NH,S)$$

**Next Variable: NA **

In [27]:
f10_FHNANHS = multiply(f3_NA, f9_FHNANHS)
f11_FHNHS = sumout(f10_FHNANHS, ['NA'])
f11_FHNHS.head()

,FH,NH,S,val
0,0,0,0,0.829929
1,0,0,1,0.411536
2,0,1,0,0.122214
3,0,1,1,0.036346
4,1,0,0,0.006857


**Updated Expression:** $$\sum_{NH} \sum_Sf_5(S) \times f_7(S) \times f_{11}(FH,NH,S)$$

**Next Variable: NH **

In [28]:
f12_FHS = sumout(f11_FHNHS, ['NH'])
f12_FHS

,FH,S,val
0,0,0,0.952143
1,0,1,0.447882
2,1,0,0.047857
3,1,1,0.552118


**Updated Expression:** $$\sum_Sf_5(S) \times f_7(S) \times f_{12}(FH,S)$$

**Next Variable: S **

In [29]:
f13_FHS = multiply(f5_S, f12_FHS)
f13_FHS

,S,FH,val
0,1,0,0.022394
1,1,1,0.027606
2,0,0,0.904536
3,0,1,0.045464


In [30]:
f14_FH = sumout(f13_FHS, ['S'])
f14_FH

,FH,val
0,0,0.92693
1,1,0.07307


Usually, normalization would be required but since in this case this is not a conditional probability distribution we get the actual proabability distribution without normalization. Below, you can see that normalization isn't required in this case but has been done for completeness

In [31]:
f15_FH = normalize(f14_FH)
f15_FH

,FH,val
0,0.0,0.92693
1,1.0,0.07307


In [32]:
print("P(FH)=", f15_FH[f15_FH['FH']==1]['val'].item())

P(FH)= 0.07307017857142856


## Q2.c

We observe fido is howling and full moon is visible in the sky. What's the probability that Fido is sick? 
1. P(S|FH and M)
2. Query Variables: S
3. Evidence Variables: FH, M
4. Hidden variables: B, NA, NH
Reusing the starting definition as last time:

$$\sum_{B} \sum_{NA} \sum_{NH} f_1(B,S) \times f_2(M) \times \times f_3(NA) \times f_4(M,NA,NH)\times f_5(S)\times  f_6(FH,M,NH,S) $$


**Step 2: Restrict the variables**

1. Fido is howling so FH = true


In [276]:
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]

In [277]:
query_2c = ['S']
hidden_2c = ['B', 'NA', 'NH']
evidence_2c = ['FH', 'M']
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
inference(factor_list,['S'], ['B','NA', 'NH'], ['FH', 'M'])

     S       val
0  1.0  0.085941
1  0.0  0.914059


In [33]:
f16_MNHS = restrict(f6_FHMNHS, variable=['FH'], value = [1])
f16_MNHS.head()

,M,NH,S,val
0,1,1,1,0.99
1,1,1,0,0.65
2,1,0,1,0.90
3,1,0,0,0.40
4,0,1,1,0.75


2. Full moon is visible so M = true

In [34]:
f17 = restrict(f2_M, ['M'], [1])
f17

,val
0,0.035714


In [35]:
f18_NANH = restrict(f4_MNANH, ['M'], [1])
f18_NANH

,NA,NH,val
0,1,1,0.8
1,1,0,0.2
2,0,1,0.4
3,0,0,0.6


In [36]:
f19_NHS = restrict(f16_MNHS, ['M'], [1])
f19_NHS

,NH,S,val
0,1,1,0.99
1,1,0,0.65
2,0,1,0.90
3,0,0,0.40


**Updated expression: **

$$\sum_{B} \sum_{NA} \sum_{NH} f_1(B,S) \times f_{17} \times f_3(NA) \times f_{18}(NA,NH)\times f_5(S)\times  f_{19}(NH,S) $$
<br/>
$$ = f_{17} \times \sum_{B} \sum_{NA} \sum_{NH} f_1(B,S) \times f_3(NA) \times f_{18}(NA,NH)\times f_5(S)\times  f_{19}(NH,S) $$

**Order for variable elimination**:
B, FH, M, NA, NH, S

**B**

In [37]:
f20_S = sumout(f1_BS, ['B'])
f20_S

,S,val
0,0,1.0
1,1,1.0


**Updated Expression:** $$ = f_{17} \times \sum_{NA} \sum_{NH} f_3(NA) \times f_{18}(NA,NH)\times f_5(S)\times  f_{19}(NH,S) \times f_{20}(S) $$

**Next Variable: NA**

In [38]:
f21_NANH = multiply(f3_NA, f18_NANH)
f21_NANH

,NA,NH,val
0,1,1,0.24
1,1,0,0.06
2,0,1,0.28
3,0,0,0.42


In [39]:
f22_NH = sumout(f21_NANH, ['NA'])
f22_NH

,NH,val
0,0,0.48
1,1,0.52


**Updated Expression:** $$ = f_{17} \times \sum_{NH} f_5(S)\times  f_{19}(NH,S) \times f_{20}(S) \times f_{22}(NH)$$

**Next Variable: NH**

In [40]:
f23_NHS = multiply(f19_NHS, f22_NH)
f23_NHS

,NH,S,val
0,1,1,0.5148
1,1,0,0.3380
2,0,1,0.4320
3,0,0,0.1920


In [41]:
f24_S = sumout(f23_NHS, ['NH'])
f24_S

,S,val
0,0,0.5300
1,1,0.9468


**Updated Expression:** $$ =f_{17} \times f_5(S) \times  f_{24}(S) \times f_{20}(S)$$
$$ = f_{17}\times f_5(S) \times  f_{24}(S) $$ because f20_S will not impact the product

Multiplying the remaining terms

In [42]:
f25_S = multiply(f5_S, f24_S)
f25_S

,S,val
0,1,0.04734
1,0,0.50350


In [43]:
f26_S = multiply(f25_S, f17)
f26_S

,S,val
0,1,0.001691
1,0,0.017982


**Final step**
We normalize the factor above to obtain the conditional probability as asked in the question

In [44]:
f27_S = normalize(f26_S)
f27_S

,S,val
0,1.0,0.085941
1,0.0,0.914059


In [45]:
print("P(S|FH and M)=", f27_S[f27_S['S']==1]['val'].item())

P(S|FH and M)= 0.08594147120761023


**Q2d**

We observe fido is howling and full moon is visible in the sky and he has not eaten his dinner. What's the probability that Fido is sick? 

1. P(S|B and FH and M)
2. Query Variables: S
3. Evidence Variables: B,FH,M
4. Hidden variables: NA, NH

Reusing the starting definition as last time by adjusting the summing variables to be only over the unobserved non query variables:

$$\sum_{NA} \sum_{NH} f_1(B,S) \times f_2(M) \times f_3(NA) \times f_4(M,NA,NH)\times f_5(S)\times  f_6(FH,M,NH,S) $$

We need to calculate $$ P(S \mid B=t \wedge FH=t \wedge M=t) $$

In [325]:
query_2d = ['S']
hidden_2d = ['NA', 'NH']
evidence_2d = ['B','FH', 'M']
factor_list = [f1_BS, f6_FHMNHS, f2_M, f4_MNANH, f3_NA, f5_S]
inference(factor_list,query_2d, hidden_2d, evidence_2d)

     S       val
0  1.0  0.360667
1  0.0  0.639333


**Step 2: Restrict the variables**

1. Fido is howling so FH = true


In [293]:
f16_MNHS = restrict(f6_FHMNHS, variable=['FH'], value = [1])
f16_MNHS

,M,NH,S,val
0,1,1,1,0.99
1,1,1,0,0.65
2,1,0,1,0.90
3,1,0,0,0.40
4,0,1,1,0.75
5,0,1,0,0.20
6,0,0,1,0.50
7,0,0,0,0.00


2. Full moon is visible so M = true

In [294]:
f17 = restrict(f2_M, ['M'], [1])
f17

,val
0,0.035714


In [295]:
f18_NANH = restrict(f4_MNANH, ['M'], [1])
f18_NANH

,NA,NH,val
0,1,1,0.8
1,1,0,0.2
2,0,1,0.4
3,0,0,0.6


In [296]:
f19_NHS = restrict(f16_MNHS, ['M'], [1])
f19_NHS

,NH,S,val
0,1,1,0.99
1,1,0,0.65
2,0,1,0.90
3,0,0,0.40


3. He did not eat dinner so B = true

In [297]:
f28_S = restrict(f1_BS, ['B'], [1])
f28_S

,S,val
0,1,0.6
1,0,0.1


**Updated expression: **

$$\sum_{NA} \sum_{NH} f_{17} \times f_3(NA) \times f_{18}(NA,NH)\times f_5(S)\times  f_{19}(NH,S) \times f_{28}(S)$$
<br/>
$$ = f_{17} \times \sum_{NA} \sum_{NH} f_3(NA) \times f_{18}(NA,NH)\times f_5(S)\times  f_{19}(NH,S) \times f_{28}(S) $$

**Order for variable elimination**:
B, FH, M, NA, NH, S

**NA**

In [298]:
f21_NANH = multiply(f3_NA, f18_NANH)
f21_NANH

,NA,NH,val
0,1,1,0.24
1,1,0,0.06
2,0,1,0.28
3,0,0,0.42


In [299]:
f22_NH = sumout(f21_NANH, ['NA'])
f22_NH

,NH,val
0,0,0.48
1,1,0.52


**Updated Expression:** $$ = f_{17} \times \sum_{NH} f_5(S)\times  f_{19}(NH,S) \times f_{22}(NH) \times f_{28}(S)$$

**Next Variable: NH**

In [300]:
f23_NHS = multiply(f19_NHS, f22_NH)
f23_NHS

,NH,S,val
0,1,1,0.5148
1,1,0,0.3380
2,0,1,0.4320
3,0,0,0.1920


In [301]:
f24_S = sumout(f23_NHS, ['NH'])
f24_S

,S,val
0,0,0.5300
1,1,0.9468


**Updated Expression**

$$ f_{17} \times f_5(S) \times f_{24}(S) \times f_{28}(S) $$

Multiplying the remaining variables together

In [302]:
f25_S = multiply(f5_S, f24_S)
f25_S

,S,val
0,1,0.04734
1,0,0.50350


In [303]:
f29_S = multiply(f25_S, f28_S)
f29_S

,S,val
0,1,0.028404
1,0,0.050350


In [304]:
f30_S = multiply(f29_S,f17)
f30_S

,S,val
0,1,0.001014
1,0,0.001798


**Final step**
We normalize the factor above to obtain the conditional probability as asked in the question

In [305]:
f31_S = normalize(f30_S)
f31_S

,S,val
0,1.0,0.360667
1,0.0,0.639333


In [227]:
print("P(S|B and FH and M)=", f31_S[f31_S['S']==1]['val'].item())

P(S|B and FH and M)= 0.36066739467201664


**Q2e**

We observe fido is howling and full moon is visible in the sky and he has not eaten his dinner and the neighbor's away. What's the probability that Fido is sick? 

Reusing the starting definition as last time by adjusting the summing variables to be only over the unobserved non query variables:

$$\sum_{NH} f_1(B,S) \times f_2(M) \times f_3(NA) \times f_4(M,NA,NH)\times f_5(S)\times  f_6(FH,M,NH,S) $$

We need to calculate $$ P(S \mid B=t \wedge FH=t \wedge M=t \wedge NA = t) $$

Step 2: Restrict the variables

1. Fido is howling so FH = true

In [60]:
f16_MNHS = restrict(f6_FHMNHS, variable=['FH'], value = [1])
f16_MNHS

,M,NH,S,val
0,1,1,1,0.99
1,1,1,0,0.65
2,1,0,1,0.90
3,1,0,0,0.40
4,0,1,1,0.75
5,0,1,0,0.20
6,0,0,1,0.50
7,0,0,0,0.00


2. Full moon is visible so M = true

In [61]:
f17 = restrict(f2_M, ['M'], [1])
f17

,val
0,0.035714


In [62]:
f18_NANH = restrict(f4_MNANH, ['M'], [1])
f18_NANH

,NA,NH,val
0,1,1,0.8
1,1,0,0.2
2,0,1,0.4
3,0,0,0.6


In [63]:
f19_NHS = restrict(f16_MNHS, ['M'], [1])
f19_NHS

,NH,S,val
0,1,1,0.99
1,1,0,0.65
2,0,1,0.90
3,0,0,0.40


3. He did not eat dinner so B = true

In [64]:
f28_S = restrict(f1_BS, ['B'], [1])
f28_S

,S,val
0,1,0.6
1,0,0.1


4. The neighbor is away so NA = true

In [65]:
f32 = restrict(f3_NA, ['NA'], [1])
f32

,val
0,0.3


In [66]:
f33_NH = restrict(f18_NANH, ['NA'], [1])
f33_NH

,NH,val
0,1,0.8
1,0,0.2


**Updated Expression**

$$ = f_{17} \times f_{32} \sum_{NH} f_{33}(NH)\times f_5(S)\times  f_{19}(NH,S) \times f_{28}(S) $$

** Summing out the only hidden variable, NH **

In [67]:
f34_NHS = multiply(f33_NH, f19_NHS)
f34_NHS

,NH,S,val
0,1,1,0.792
1,1,0,0.520
2,0,1,0.180
3,0,0,0.080


In [68]:
f35_S = sumout(f34_NHS, ['NH'])
f35_S

,S,val
0,0,0.600
1,1,0.972


**Updated Expression**

$$ = f_{17} \times f_{32} \times f_5(S) \times f_{35}(S) \times f_{28}(S) $$

In [69]:
f36_S = multiply(f5_S, f35_S)
f36_S

,S,val
0,1,0.0486
1,0,0.5700


In [70]:
f37_S = multiply(f36_S, f28_S)
f37_S

,S,val
0,1,0.02916
1,0,0.05700


In [71]:
f38_S = multiply(f37_S, f17)
f38_S

,S,val
0,1,0.001041
1,0,0.002036


In [72]:
f39_S = multiply(f38_S, f32)
f39_S

,S,val
0,1,0.000312
1,0,0.000611


**Final step**
We normalize the factor above to obtain the conditional probability as asked in the question

In [73]:
f40_S = normalize(f39_S)
f40_S

,S,val
0,1.0,0.33844
1,0.0,0.66156


In [74]:
print("P(S|B and FH and M and NA)=", f31_S[f31_S['S']==1]['val'].item())

P(S|B and FH and M and NA)= 0.36066739467201664


# Exercise 2

**TL DR;**
1. P(G|W)= 0.1346153846153846
2. P(G|W')= 0.06766055045871562
3. P(B|W and G and A)= 0.6563636363636363
4. P(B|A)= 0.6563636363636363
5. P(B|W)= 0.16725274725274725
6. P(E|A and B)= 0.052631578947368425
7. P(E|A)= 0.06727272727272726

Query Variable = G <br/>
Evidence Variable = W <br/>
Hidden Variables = A,B,E <br/>

Variable Elimination order : W, G, A, B, E

$$P(G\mid W)$$
can be rewritten as
$$P(G\mid W=t \wedge A=a \wedge B=b \wedge E=e)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_A \sum_B \sum_E P(B=b) P(E=e) P(A=a \mid B=b \wedge E=e) P(W = t \mid A=a) P(G=g \mid A=a)$$ <br/><br/><br/>

**Step 1: Define factors **


Definining a factor $$f_1(B) = P(B=b) $$

In [75]:
f1_B = pd.DataFrame([[1,0.1], [0, 0.9]], columns = ['B', 'val'])
f1_B 

,B,val
0,1,0.1
1,0,0.9


Defining a factor $$f_2(E) = P(E=e)$$

In [76]:
f2_E = pd.DataFrame([[1,0.05], [0,0.95]], columns = ['E', 'val'])
f2_E

,E,val
0,1,0.05
1,0,0.95


Defining a factor $$ f_3(A,B,E) = P( A=a \mid B=b \wedge E=e)$$

In [77]:
f3_ABE = pd.DataFrame([[1,1,1,0.95], 
                       [1,1,0,0.90], 
                       [1,0,1,0.10], 
                       [1,0,0,0.05], 
                       [0,1,1,0.05],
                       [0,1,0,0.10],
                       [0,0,1,0.90],
                       [0,0,0,0.95]], columns = ['A', 'B', 'E', 'val'])
f3_ABE

,A,B,E,val
0,1,1,1,0.95
1,1,1,0,0.90
2,1,0,1,0.10
3,1,0,0,0.05
4,0,1,1,0.05
5,0,1,0,0.10
6,0,0,1,0.90
7,0,0,0,0.95


Defining a factor $$f_4(A,W) = P(W=t\mid A=a)$$

In [78]:
f4_AW = pd.DataFrame([[1,1,0.8],
                     [1,0,0.2], 
                     [0,1,0.4],
                     [0,0,0.6]], columns = ['A', 'W', 'val'])
f4_AW

,A,W,val
0,1,1,0.8
1,1,0,0.2
2,0,1,0.4
3,0,0,0.6


Defining a factor $$f_5(A,G) = P(G=g\mid A=a)$$

In [79]:
f5_AG = pd.DataFrame([[1,1,0.4],
                     [1,0,0.6], 
                     [0,1,0.05],
                     [0,0,0.95]], columns = ['A', 'G', 'val'])
f5_AG

,A,G,val
0,1,1,0.40
1,1,0,0.60
2,0,1,0.05
3,0,0,0.95


**Expression using factors**:

$$ \sum_A \sum_B \sum_E f_1(B) \times f_2(E) \times f_3(A,B,E) \times f_5(A,G) \times f_4(A,W) $$ 

**Step 2: Restrict the variables**

Restricting the variable f4 with W = true

In [80]:
f6_A = restrict(f4_AW, ['W'], [1])
f6_A

,A,val
0,1,0.8
2,0,0.4


** Updated expression ** 

$$ \sum_A \sum_B \sum_E f_1(B) \times f_2(E) \times f_3(A,B,E) \times f_5(A,G) \times f_6(A) $$ 

**Step 3: Sum out hidden variables according to an order**

**Next Variable: A**
Factors relevant: f3, f5, f6

In [81]:
f7_ABEG_temp = multiply(f3_ABE, f5_AG)
f7_ABEG_temp.head()

,A,B,E,G,val
0,1,1,1,1,0.38
1,1,1,1,0,0.57
2,1,1,0,1,0.36
3,1,1,0,0,0.54
4,1,0,1,1,0.04


In [82]:
f7_ABEG = multiply(f7_ABEG_temp, f6_A)
f7_ABEG.head()

,A,B,E,G,val
0,1,1,1,1,0.304
1,1,1,1,0,0.456
2,1,1,0,1,0.288
3,1,1,0,0,0.432
4,1,0,1,1,0.032


In [83]:
f8_BEG = sumout(f7_ABEG, ['A'])
f8_BEG

,B,E,G,val
0,0,0,0,0.385
1,0,0,1,0.035
2,0,1,0,0.390
3,0,1,1,0.050
4,1,0,0,0.470
5,1,0,1,0.290
6,1,1,0,0.475
7,1,1,1,0.305


** Updated expression ** 

$$ \sum_B \sum_E f_1(B) \times f_2(E) \times f_8(B,E,G) $$ 

**Next Variable: B** <br/>
Factors relevant: f1, f8

In [84]:
f9_BEG = multiply(f1_B, f8_BEG)
f9_BEG

,B,E,G,val
0,1,0,0,0.0470
1,1,0,1,0.0290
2,1,1,0,0.0475
3,1,1,1,0.0305
4,0,0,0,0.3465
5,0,0,1,0.0315
6,0,1,0,0.3510
7,0,1,1,0.0450


In [85]:
f10_EG = sumout(f9_BEG, ['B'])
f10_EG

,E,G,val
0,0,0,0.3935
1,0,1,0.0605
2,1,0,0.3985
3,1,1,0.0755


** Updated expression ** 

$$\sum_E f_2(E) \times f_{10}(E,G) $$ 

**Next Variable: E** <br/>
Factors relevant: f2, f10

In [86]:
f11_EG = multiply(f2_E, f10_EG)
f11_EG

,E,G,val
0,1,0,0.019925
1,1,1,0.003775
2,0,0,0.373825
3,0,1,0.057475


In [87]:
f12_G = sumout(f11_EG, ['E'])
f12_G

,G,val
0,0,0.39375
1,1,0.06125


**Final step: Normalize**

In [88]:
f13_G = normalize(f12_G)
f13_G

,G,val
0,0.0,0.865385
1,1.0,0.134615


In [89]:
print("P(G|W)=", f13_G[f13_G["G"]==1]["val"].item())

P(G|W)= 0.1346153846153846


##### To Calculate P(G|W') we use the same starting expression from above and restrict W to false

**Expression using factors**:

$$ \sum_A \sum_B \sum_E f_1(B) \times f_2(E) \times f_3(A,B,E) \times f_5(A,G) \times f_4(A,W) $$ 

In [90]:
f14_A = restrict(f4_AW, ['W'], [0])
f14_A

,A,val
1,1,0.2
3,0,0.6


** Updated expression ** 

$$ \sum_A \sum_B \sum_E f_1(B) \times f_2(E) \times f_3(A,B,E) \times f_5(A,G) \times f_{14}(A) $$ 

**Step 3: Sum out hidden variables according to an order**

**Next Variable: A**
Factors relevant: f3, f5, f14

In [91]:
f7_ABEG_temp = multiply(f3_ABE, f5_AG)
f15_ABEG = multiply(f7_ABEG_temp, f14_A)
f15_ABEG.head()

,A,B,E,G,val
0,1,1,1,1,0.076
1,1,1,1,0,0.114
2,1,1,0,1,0.072
3,1,1,0,0,0.108
4,1,0,1,1,0.008


In [92]:
f16_BEG = sumout(f15_ABEG, ['A'])
f16_BEG

,B,E,G,val
0,0,0,0,0.5475
1,0,0,1,0.0325
2,0,1,0,0.5250
3,0,1,1,0.0350
4,1,0,0,0.1650
5,1,0,1,0.0750
6,1,1,0,0.1425
7,1,1,1,0.0775


** Updated expression ** 

$$ \sum_B \sum_E f_1(B) \times f_2(E) \times f_{16}(B,E,G) $$ 

**Next Variable: B** <br/>
Factors relevant: f1, f16

In [93]:
f17_BEG = multiply(f1_B, f16_BEG)
f17_BEG

,B,E,G,val
0,1,0,0,0.01650
1,1,0,1,0.00750
2,1,1,0,0.01425
3,1,1,1,0.00775
4,0,0,0,0.49275
5,0,0,1,0.02925
6,0,1,0,0.47250
7,0,1,1,0.03150


In [94]:
f18_EG = sumout(f17_BEG, ['B'])
f18_EG

,E,G,val
0,0,0,0.50925
1,0,1,0.03675
2,1,0,0.48675
3,1,1,0.03925


** Updated expression ** 

$$\sum_E f_2(E) \times f_{18}(E,G) $$ 

**Next Variable: E** <br/>
Factors relevant: f2, f18

In [95]:
f19_EG = multiply(f2_E, f18_EG)
f19_EG

,E,G,val
0,1,0,0.024338
1,1,1,0.001963
2,0,0,0.483787
3,0,1,0.034913


In [96]:
f20_G = sumout(f19_EG, ['E'])
f20_G

,G,val
0,0,0.508125
1,1,0.036875


**Final step: Normalize**

In [97]:
f21_G = normalize(f20_G)
f21_G

,G,val
0,0.0,0.932339
1,1.0,0.067661


In [98]:
print("P(G|W')=", f21_G[f21_G["G"]==1]["val"].item())

P(G|W')= 0.06766055045871562


##### We can see that P(G|W) is not equal to P(G|W'); Hence Proved! 

2. Calculate P(B|W and G and A) and P(B|A). Show that they're the same

**P(B|W and G and A) **<br/>
Query Variable = B <br/>
Evidence Variable = W,G,A <br/>
Hidden Variables = E <br/>

Variable Elimination order : W, G, A, B, E

$$P(B\mid A \wedge G \wedge W)$$
can be rewritten as
$$P(B\mid A=t \wedge E=e \wedge G=t \wedge W=t)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_E P(B=b) P(E=e) P(A=t \mid B=b \wedge E=e) P(W = t \mid A=t) P(G=t \mid A=t)$$

We'll reuse the factors defined above to avoid redundancy but I will be listing them below for completeness 

In [99]:
# For P(B=b)
f1_B

,B,val
0,1,0.1
1,0,0.9


In [100]:
# For P(E=e)
f2_E

,E,val
0,1,0.05
1,0,0.95


In [101]:
# For P(A=t|B=b and E=e)
f3_ABE

,A,B,E,val
0,1,1,1,0.95
1,1,1,0,0.90
2,1,0,1,0.10
3,1,0,0,0.05
4,0,1,1,0.05
5,0,1,0,0.10
6,0,0,1,0.90
7,0,0,0,0.95


In [102]:
# For P(W=t|A=t)
f4_AW

,A,W,val
0,1,1,0.8
1,1,0,0.2
2,0,1,0.4
3,0,0,0.6


In [103]:
# For P(G=t|A=t)
f5_AG

,A,G,val
0,1,1,0.40
1,1,0,0.60
2,0,1,0.05
3,0,0,0.95


**Step 2: Restrict factors**
We observe more variables than last itme so the restriction of variables will change impacting the calculation.<br/>
Specificially: A, G, W are the evidence variables with all of them being true <br/>
Starting with A: relevant factors are f3_ABE, f4_AW and f5_AG

In [104]:
f22_BE = restrict(f3_ABE, ['A'], [1])
f22_BE

,B,E,val
0,1,1,0.95
1,1,0,0.90
2,0,1,0.10
3,0,0,0.05


In [105]:
f23_W = restrict(f4_AW, ['A'], [1])
f23_W

,W,val
0,1,0.8
1,0,0.2


In [106]:
f24_G = restrict(f5_AG, ['A'], [1])
f24_G

,G,val
0,1,0.4
1,0,0.6


In [107]:
# Next we restrict G to be true. Relevant factors include f24_G
f25 = restrict(f24_G, ['G'], [1])
f25

,val
0,0.4


In [108]:
# Next we restrict W to be true. Relevant factors include f23_W
f26 = restrict(f23_W, ['W'],[1])
f26

,val
0,0.8


**Updated Expression:**

$$\sum_E f_1(B) \times f_2(E) \times f_{22}(BE) \times f_{26} \times f_{25}$$

<br/> Summing out E which is the only hidden variable in this case. Relevant factors include f2(E), f22(BE)

In [109]:
f27_BE = multiply(f2_E, f22_BE)
f27_BE

,E,B,val
0,1,1,0.0475
1,1,0,0.0050
2,0,1,0.8550
3,0,0,0.0475


In [110]:
f28_B = sumout(f27_BE, ['E'])
f28_B

,B,val
0,0,0.0525
1,1,0.9025


**Updated Expression:**

$$f_1(B) \times f_{28}(B) \times f_{26} \times f_{25}$$

In [111]:
f29_B = multiply(f1_B, f28_B)
f29_B

,B,val
0,1,0.09025
1,0,0.04725


In [112]:
f30_B = multiply(f29_B, f26)
f31_B = multiply(f30_B, f25)
f31_B

,B,val
0,1,0.02888
1,0,0.01512


**Final step: normalization**

In [113]:
f32_B = normalize(f31_B)
f32_B

,B,val
0,1.0,0.656364
1,0.0,0.343636


In [114]:
print("P(B|W and G and A)=", f32_B[f32_B["B"]==1]["val"].item())

P(B|W and G and A)= 0.6563636363636364


2. Calculate P(B|W and G and A) and P(B|A). Show that they're the same

**P(B|A) **<br/>
Query Variable = B <br/>
Evidence Variable = A <br/>
Hidden Variables = E,G,W <br/>

Variable Elimination order : W, G, A, B, E

$$P(B\mid A)$$
can be rewritten as
$$P(B\mid A=t \wedge E=e \wedge G=g \wedge W=w)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_W \sum_G \sum_E P(B=b) P(E=e) P(A=t \mid B=b \wedge E=e) P(W = w \mid A=t) P(G=g \mid A=t)$$

I'll reuse the factors defined above to avoid redundancy but I will be listing them below for completeness 

In [115]:
# For P(B=b) f1_B
# For P(E=e) f2_E
# For P(A=t|B=b and E=e) f3_ABE
# For P(W=w|A=t) f4_AW
# For P(G=g|A=t) f5_AG

In [116]:
# Restrict the only evidence variable: A
f22_BE = restrict(f3_ABE, ['A'], [1])
f22_BE

,B,E,val
0,1,1,0.95
1,1,0,0.90
2,0,1,0.10
3,0,0,0.05


In [117]:
f23_W = restrict(f4_AW, ['A'], [1])
f23_W

,W,val
0,1,0.8
1,0,0.2


In [118]:
f24_G = restrict(f5_AG, ['A'], [1])
f24_G

,G,val
0,1,0.4
1,0,0.6


**Updated Expression**

$$\sum_W \sum_G \sum_E f_1(B) \times f_2(E) \times f_{22}(BE) \times f_{23}(W) \times f_{24}(G) $$

**Step 3: Summing out hidden variables**
1. W with factors f23(W)

In [119]:
f33_W = sumout(f23_W, ['W'])
# 2. G wth factors f24(G)
f34_G = sumout(f24_G, ['G'])

In [120]:
# E with factors f2(E) and f22(BE)
f27_BE = multiply(f2_E, f22_BE)
f28_B = sumout(f27_BE, ['E'])

**Step 4: Multiply remaining factors**
f1_B, f28_B, f33_w, f34_G

In [121]:
f35 = multiply(f1_B, f28_B)
f36 = multiply(f35, f33_W)
f37 = multiply(f36, f34_G)
f37

,B,val
0,1,0.09025
1,0,0.04725


**Step 5: Normalize the factor**

In [122]:
f38_B = normalize(f37)
f38_B

,B,val
0,1.0,0.656364
1,0.0,0.343636


In [123]:
print("P(B|A)=", f38_B[f38_B["B"]==1]["val"].item())

P(B|A)= 0.6563636363636363


##### The two values obtained are the same. Hence Proved! 

3. We could not have chosen W as the only parent of B because B is not conditionally independent of A and G given W. Calculate P(B|A ∧ G ∧ W) and P(B|W) separately and show that they are not equal.

Since we calculated P(B|A and G and W) in the previous part. We only need to prove that P(B|W) is **not** equal to 0.6563636363636363

**P(B|W) **<br/>
Query Variable = B <br/>
Evidence Variable = W <br/>
Hidden Variables = A,E,G <br/>

Variable Elimination order : W, G, A, B, E

$$P(B\mid W)$$
can be rewritten as
$$P(B\mid A=a \wedge E=e \wedge G=g \wedge W=t)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_G \sum_A \sum_E P(B=b) P(E=e) P(A=a \mid B=b \wedge E=e) P(W = t \mid A=a) P(G=g \mid A=a)$$

I'll reuse the factors defined above to avoid redundancy but I will be listing them below for completeness 

In [124]:
# For P(B=b) f1_B
# For P(E=e) f2_E
# For P(A=a|B=b and E=e) f3_ABE
# For P(W=t|A=a) f4_AW
# For P(G=g|A=a) f5_AG

In [125]:
# Restrict the only evidence variable: W
f39_A = restrict(f4_AW, ['W'], [1])
f39_A

,A,val
0,1,0.8
2,0,0.4


$$\sum_G \sum_A \sum_E f_1(B) \times f_2(E) \times f_{3}(ABE) \times f_{39}(A) \times f_{5}(AG) $$

<br/>
**Step 3: Summing out hidden variables**

Next Variable: G, factors: f5_AG

In [126]:
f40_A = sumout(f5_AG, ['G'])
f40_A

,A,val
0,0,1.0
1,1,1.0


Updated Expression: <br/>
$$\sum_A \sum_E f_1(B) \times f_2(E) \times f_{3}(ABE) \times f_{39}(A) \times f_{40}(A) $$

Next Variable:A, factors: f3, f39, f40

In [127]:
f41_ABE = multiply(f3_ABE, f39_A)
f41_ABE = multiply(f41_ABE, f40_A)
f41_BE = sumout(f41_ABE, ['A'])
f41_BE

,B,E,val
0,0,0,0.42
1,0,1,0.44
2,1,0,0.76
3,1,1,0.78


Updated Expression: <br/>
$$\sum_E f_1(B) \times f_2(E) \times f_{41}(BE) $$

Next Variable:E, factors: f2, f41

In [128]:
f42_BE = multiply(f41_BE, f2_E)
f42_B = sumout(f42_BE, ['E'])
f42_B

,B,val
0,0,0.421
1,1,0.761


Updated Expression: <br/>
$$f_1(B) \times f_{42}(B) $$

Next step: multiply the two factors

In [129]:
f43_B = multiply(f42_B, f1_B)
f43_B

,B,val
0,0,0.3789
1,1,0.0761


In [130]:
# Normalize it
f44_B = normalize(f43_B)
f44_B

,B,val
0,0.0,0.832747
1,1.0,0.167253


In [131]:
print("P(B|W)=", f44_B[f44_B["B"]==1]["val"].item())

P(B|W)= 0.16725274725274725


##### As we can see the values differ, hence proved! 

4. We could not have chosen A as the only parent of E because E is not conditionally independent of B given A. Calculate P(E|A∧B) and P(E|A) separately and show that they are not equal.

**P(E|A and B) **<br/>
Query Variable = E <br/>
Evidence Variable = A,B <br/>
Hidden Variables = W,G <br/>

Variable Elimination order : W, G, A, B, E

$$P(E\mid A \wedge B)$$
can be rewritten as
$$P(E\mid A=t \wedge B=t \wedge G=g \wedge W=w)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_W \sum_G P(B=t) P(E=e) P(A=t \mid B=t \wedge E=e) P(W = w \mid A=t) P(G=g \mid A=t)$$

I'll reuse the factors defined above to avoid redundancy but I will be listing them below for completeness 

In [132]:
# For P(B=t) f1_B
# For P(E=e) f2_E
# For P(A=t|B=t and E=e) f3_ABE
# For P(W=w|A=t) f4_AW
# For P(G=g|A=t) f5_AG

In [133]:
# Restrict A = t and B=t
f45_ABE = restrict(f3_ABE, ['A'], [1])
f46_AW = restrict(f4_AW, ['A'], [1])
f47_AG = restrict(f5_AG, ['A'], [1])
f48_B = restrict(f1_B, ['B'], [1])
f49_ABE = restrict(f45_ABE, ['B'], [1])

Updated Expression:

$$\sum_W \sum_G f_{48}(B) \times f_2(E) \times f_{49}(ABE) \times f_{46}(AW) \times f_{47}(AG)$$

Next step to sum out the hidden variables. 
First, we take W based on the order and the relevant factors are: f46_AW and then G with f47_AG

In [134]:
f50_A = sumout(f46_AW, ['W'])
f51_A = sumout(f47_AG, ['G'])

$$ f_{48}(B) \times f_2(E) \times f_{49}(ABE) \times f_{50}(A) \times f_{51}(A)$$

Next step: we multiply everything and normalize

In [135]:
f52_ABE = multiply(f49_ABE, f50_A)
f52_ABE = multiply(f52_ABE, f51_A)
f53_ABE = multiply(f52_ABE, f48_B)
f54_E = multiply(f53_ABE, f2_E)
f54_E

,E,val
0,1,0.00475
1,0,0.08550


In [136]:
f55_E = normalize(f54_E)
f55_E

,E,val
0,1.0,0.052632
1,0.0,0.947368


In [137]:
print("P(E|A and B)=", f55_E[f55_E["E"]==1]["val"].item())

P(E|A and B)= 0.052631578947368425


**P(E|A) **<br/>
Query Variable = E <br/>
Evidence Variable = A <br/>
Hidden Variables = W,G, B <br/>

Variable Elimination order : W, G, A, B, E

$$P(E\mid A)$$
can be rewritten as
$$P(E\mid A=t \wedge B=b \wedge G=g \wedge W=w)$$

using the bayesian network relations from Figure 1, the above equation can be broken down as: <br/><br/>
$$\sum_W \sum_G \sum_B P(B=b) P(E=e) P(A=t \mid B=b \wedge E=e) P(W = w \mid A=t) P(G=g \mid A=t)$$

I'll reuse the factors defined above to avoid redundancy but I will be listing them below for completeness 

In [138]:
# For P(B=b) f1_B
# For P(E=e) f2_E
# For P(A=t|B=b and E=e) f3_ABE
# For P(W=w|A=t) f4_AW
# For P(G=g|A=t) f5_AG

In [139]:
# Restrict A = t
f45_ABE = restrict(f3_ABE, ['A'], [1])
f46_AW = restrict(f4_AW, ['A'], [1])
f47_AG = restrict(f5_AG, ['A'], [1])

Updated Expression:

$$\sum_W \sum_G \sum_B f_{1}(B) \times f_2(E) \times f_{45}(ABE) \times f_{46}(AW) \times f_{47}(AG)$$

Next step to sum out the hidden variables. 
First, we take W based on the order and the relevant factors are: f46_AW and then G with f47_AG

In [140]:
f50_A = sumout(f46_AW, ['W'])
f51_A = sumout(f47_AG, ['G'])

In [141]:
# Next we take B with factors f1_B, f45_ABE
f56_BE = multiply(f1_B, f45_ABE)
f57_E = sumout(f56_BE, ['B'])
f57_E

,E,val
0,0,0.135
1,1,0.185


$$ f_{57}(E) \times f_2(E) \times f_{50}(A) \times f_{51}(A)$$

In [142]:
f58_E = multiply(f57_E, f2_E)
f58_E = multiply(f58_E, f50_A)
f58_E = multiply(f58_E, f51_A)
f58_E = normalize(f58_E)
f58_E

,E,val
0,0.0,0.932727
1,1.0,0.067273


In [143]:
print("P(E|A)=", f58_E[f58_E["E"]==1]["val"].item())

P(E|A)= 0.06727272727272726


##### P(E|A and B) is not equal to P(E|A). E is not conditionally independent of B given A. 